In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from passwords import *
import boto3

### Constants

In [2]:
str_project = os.getcwd().split('\\')[-1].replace('_', '-')
str_step = 'input'

print(f'Project is: {str_project}')
print(f'step is: {str_step}')

Project is: 20240418-loan-portfolio-app
step is: input


### Functions

In [3]:
# upload to s3
def upload_to_s3(aws_access_key_id, aws_secret_access_key, str_local_path, str_bucket_key, str_bucket_name, aws_session_token=None):
    # init client
    cls_client = boto3.client(
        's3',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        aws_session_token=aws_session_token,
    )
    # upload
    cls_client.upload_file(
        str_local_path, 
        str_bucket_name, 
        str_bucket_key,
    )

### Read in input

In [4]:
df = pd.read_csv('./01_analysis/sql/df_credit_risk.csv')
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


### Write to s3

In [5]:
# write to local csv
str_filename = 'df_credit_risk.gzip'
str_local_path = f'{str_filename}'
df.to_parquet(
    str_local_path,
    index=False,
    compression='gzip'
)

# upload to s3
str_bucket_key = f'{str_step}/{str_filename}'
upload_to_s3(
    aws_access_key_id=AWS_ACCESS_KEY_ID, 
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY, 
    str_local_path=str_local_path, 
    str_bucket_key=str_bucket_key, 
    str_bucket_name=str_project, 
)

### Remove local path

In [6]:
os.remove(str_filename)